# Semester 1 Project Submission



## Elliot

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## Ian

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## Frida

In [21]:
# Importing
import pandas as pd

In [22]:
# Reading the "Movie info" file
movie_info_df = pd.read_csv('databases/rt.movie_info.tsv', sep='\t', encoding = 'utf-8', index_col=0)
movie_info_df.head()

,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
id,,,,,,,,,,,
1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


### Data Cleaning

In [18]:
# Data frame overview. Column information, name-data type.
movie_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1560 entries, 1 to 2000
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   synopsis      1498 non-null   object
 1   rating        1557 non-null   object
 2   genre         1552 non-null   object
 3   director      1361 non-null   object
 4   writer        1111 non-null   object
 5   theater_date  1201 non-null   object
 6   dvd_date      1201 non-null   object
 7   currency      340 non-null    object
 8   box_office    340 non-null    object
 9   runtime       1530 non-null   object
 10  studio        494 non-null    object
dtypes: object(11)
memory usage: 146.2+ KB


In [3]:
# Column information, name-data type.
movie_info_df.describe()

,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
count,1498,1557,1552,1361,1111,1201,1201,340,340,1530,494
unique,1497,6,299,1125,1069,1025,717,1,336,142,200
top,A group of air crash survivors are stranded in...,R,Drama,Steven Spielberg,Woody Allen,"Jan 1, 1987","Jun 1, 2004",$,"600,000",90 minutes,Universal Pictures
freq,2,521,151,10,4,8,11,340,2,72,35


In [23]:
# Missing values
movie_info_df.isna().sum()

synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [24]:
# Removing duplicate records
movie_info_df = movie_info_df.drop_duplicates()

In [30]:
# Changing NaN values for String "Not Available"
movie_info_df['synopsis'] = movie_info_df['synopsis'].fillna('Not Available')
movie_info_df['rating'] = movie_info_df['rating'].fillna('Not Available')
movie_info_df['genre'] = movie_info_df['genre'].fillna('Not Available')
movie_info_df['director'] = movie_info_df['director'].fillna('Not Available')
movie_info_df['theater_date'] = movie_info_df['theater_date'].fillna('Not Available')
movie_info_df['dvd_date'] = movie_info_df['dvd_date'].fillna('Not Available')
movie_info_df['runtime'] = movie_info_df['runtime'].fillna('Not Available')
movie_info_df['studio'] = movie_info_df['studio'].fillna('Not Available')
movie_info_df['writer'] = movie_info_df['writer'].fillna('Not Available')

In [31]:
# Changing NaN values for 0 in Box Office column
movie_info_df['box_office'] = movie_info_df['box_office'].fillna(0)

# Remmoving commas from Box Office values
movie_info_df['box_office'] = movie_info_df['box_office'].replace({',':''}, regex=True)

# Converting Box Office values into Integers
movie_info_df['box_office'] = movie_info_df['box_office'].astype(float)

synopsis        0
rating          0
genre           0
director        0
writer          0
theater_date    0
dvd_date        0
currency        0
box_office      0
runtime         0
studio          0
dtype: int64

In [ ]:
# Checking possible Currency values
currency_values = set(movie_info_df.loc[:,"currency"])
currency_values

# Changing NaN values for "$" in Currency column
movie_info_df['currency'] = movie_info_df['currency'].fillna('$')

# Confirming that there in no more NaN values 
movie_info_df.isna().sum()

In [10]:
# Movies with highest earnings
box_o = movie_info_df.sort_values('box_office')
box_o.head(5)

,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
id,,,,,,,,,,,
108,A picture-perfect family moves into an upscale...,R,Comedy|Drama|Kids and Family,Derrick Borte,Derrick Borte,"Apr 16, 2010","Aug 10, 2010",$,"1,000,000",93 minutes,Roadside Attractions
22,Two-time Academy Award Winner Kevin Spacey giv...,R,Comedy|Drama|Mystery and Suspense,George Hickenlooper,Norman Snider,"Dec 17, 2010","Apr 5, 2011",$,"1,039,869",108 minutes,ATO Pictures
1046,Writer/director Rodrigo Garca (Nine Lives) tea...,R,Drama,NaN,NaN,"May 7, 2010","Dec 14, 2010",$,"1,110,286",126 minutes,Sony Pictures Classics
1398,"A heartfelt tale of inspiration, hope and rede...",PG,Drama|Kids and Family|Faith and Spirituality,David Nixon|Patrick Doughtie,NaN,"Apr 9, 2010","Aug 10, 2010",$,"1,121,000",110 minutes,Vivendi Entertainment
1564,Real-life sweethearts Michael Cera and Charlyn...,PG-13,Comedy,Nicholas Jasenovec,Nicholas Jasenovec|Charlyne Yi,"Aug 14, 2009","Dec 1, 2009",$,"1,149,350",89 minutes,Overture Films


In [14]:
#type(movie_info_df.loc[1,"theater_date"])
m = movie_info_df["theater_date"].str.split()
m

id
1        [Oct, 9,, 1971]
3       [Aug, 17,, 2012]
5       [Sep, 13,, 1996]
6        [Dec, 9,, 1994]
7                    NaN
              ...       
1996    [Aug, 18,, 2006]
1997    [Jul, 23,, 1993]
1998     [Jan, 1,, 1962]
1999     [Apr, 1,, 1993]
2000    [Sep, 27,, 2001]
Name: theater_date, Length: 1560, dtype: object

## Enrique

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## John

In [ ]:
# Your code here - remember to use markdown cells for comments as well!